<a href="https://colab.research.google.com/github/Mefoolyhi/JokeGenieBot/blob/main/faiss_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Encode docs using sentence-transformer

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

total_data = pd.read_csv('total_data_jokes_512.csv', encoding = "utf-8")
total_data

,joke
0,"Только заметил, что слово ""п@рно"" набирается с..."
1,"Друзья мои, чтобы соответствовать вам, я готов..."
2,"- Люся, ты все еще хранишь мой подарок?\n- Да...."
3,"- А вот скажи честно, ты во сне храпишь?\n- По..."
4,"Поссорилась с мужем. Пока он спал, я мысленно ..."
...,...
170284,- Мы на острове! Надо выбираться! Давайте пере...
170285,"- Здравствуйте, а где вход на нудистский пляж?..."
170286,"Две старушки.\n- Никитична, у тебя какое миним..."
170287,"- Ты опять был у своей шлюхи?\n- Дорогая, надо..."


In [ ]:
encoded_data = encoder.encode(total_data['joke'])

## Add to index

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.2 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np

# IndexFlatIP: Flat inner product (for small datasets)
# IndexIDMap: store document ids in the index as well
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.arange(len(total_data)))

## Search for question

In [ ]:
jokes = total_data['joke']

In [ ]:
last_liked = [jokes[345], jokes[113], jokes[4556], jokes[670], jokes[123456]] #от 1 до 5, пять последних лайкнутых (или меньше если их меньше)
query_vector = encoder.encode(last_liked)
top_k = index.search(query_vector, 10)
top_jokes = []
for _id in top_k[1][0]:
    print(_id)
    print(jokes[_id])
    print()
    top_jokes.append(jokes[_id])



345
Духовные лидеры мусульман России обратились к разработчикам лекарств от коронавируса с просьбой раскрыть состав вакцин. Похоже, имамы всерьез подозревают, нет ли в них свинины.

10504
Премьер Мишустин заявил об изменениях в работе курортов в случае прихода второй волны. Основным местом отдыха россиян станет быстровозводимый зонаторий с упругими стенами и системой перепуска взбесившихся в нагаечную комнату и палочную смирильню.

88315
Доброе утро, уважаемые радиослушатели!Вы только что прослушали передачу о безопасности приближающейся к нашей Галактике черной дыры.В Москве по-прежнему 7 часов 51 минута...

46952
По роду деятельности регулярно общаюсь с китайскими товарищами. В силу менталитета и своеобразного отношения к русскому языку регулярно выдают перлы. Из сегодняшнего:
- Мы вам выслали инструкцию. Она на китайском, но там все понятно...

91173
В Москве было произведено вооруженное ограбление ресторана. Через несколько минут после преступления свидетели видели выходящего из ре

## Save and load index

In [ ]:
path = './faiss.index'

# Save index
faiss.write_index(index, path)

In [ ]:
unliked = [jokes[3456], jokes[1143], jokes[45156], jokes[6750], jokes[1236]] # все которым он поставил дизлайк

In [ ]:
from nltk.tokenize import NLTKWordTokenizer

tokenizer = NLTKWordTokenizer()
unliked_tok = [tokenizer.tokenize(x.lower()) for x in unliked]
top_jokes_tok = [tokenizer.tokenize(x.lower()) for x in top_jokes]

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

russian_stopwords = stopwords.words("russian")
unliked_tokens = []
for row in unliked_tok:
    row_tokens = []
    for token in row:
        if token not in russian_stopwords and token not in punctuation:
            row_tokens.append(lemmatizer.lemmatize(token))
    unliked_tokens.append(' '.join(row_tokens))

top_tokens = []
for row in top_jokes_tok:
    row_tokens = []
    for token in row:
        if token not in russian_stopwords and token not in punctuation:
            row_tokens.append(lemmatizer.lemmatize(token))
    top_tokens.append(row_tokens)

In [ ]:
unliked_tokens

['парень ведет машину левой рукой рулит правой — подружку обнимает. гаишник перекрестке кричит двумя руками рулить козел',
 'тебе плохо — сильный мужчина поможет слабый сделает вид хуже ... петрович хорош п здеть пианино дотащим',
 "опытная гадалка посетители звонят дверь задает вопрос `` '' сразу говорит `` ''",
 'лягушка прыгает дороге. сзади наезжает асфальтоукладчик вдавливает асфальт. лягушка отлепляется асфальта прижал прижал наши мужики болоте',
 'это витас новую песню сочинил это преснякову яйца лифте прищемило']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer()
unliked_array = tf_idf.fit_transform(unliked_tokens)
threshold = 0.2
important_words = [word for word, score in zip(tf_idf.get_feature_names_out(), unliked_array.toarray()[0]) if score >= threshold]
print(important_words)

['ведет', 'гаишник', 'двумя', 'козел', 'кричит', 'левой', 'машину', 'обнимает', 'парень', 'перекрестке', 'подружку', 'правой', 'руками', 'рукой', 'рулит', 'рулить']


In [ ]:
important_set = set(important_words)
counts = []
for joke in top_tokens:
    local_count = 0
    for token in joke:
        if token in important_words:
            local_count += 1
    counts.append(local_count)

print(top_jokes[np.argmin(counts)])

Духовные лидеры мусульман России обратились к разработчикам лекарств от коронавируса с просьбой раскрыть состав вакцин. Похоже, имамы всерьез подозревают, нет ли в них свинины.


# Код в бота

Обязательно надо быть на GPU!

In [ ]:
# Перед запуском бота

!pip install faiss-gpu
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer
import nltk
import faiss
import numpy as np
import pandas as pd

# это вместо load_jokes
total_data = pd.read_csv('total_data_jokes_512.csv', encoding = "utf-8")
jokes = total_data['joke']

encoder = SentenceTransformer('bert-base-nli-mean-tokens')
nltk.download('stopwords')
nltk.download('wordnet')
encoded_data = encoder.encode(jokes) # это будет работать минут 10, это нормально
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.arange(len(jokes)))

In [ ]:
# При генерации шутки
from nltk.tokenize import NLTKWordTokenizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# тут ищем 10 самых похожих на последние понравившиеся
last_liked = [jokes[345], jokes[113], jokes[4556], jokes[670], jokes[123456]] #от 1 до 5 последних лайкнутых шуток (5 или меньше если их меньше)
# если у человека нет истории, то просто даем любую шутку
query_vector = encoder.encode(last_liked)
top_k = index.search(query_vector, 10)
top_jokes = []
for _id in top_k[1][0]:
    top_jokes.append(jokes[_id])

unliked = [jokes[3456], jokes[1143], jokes[45156], jokes[6750], jokes[1236]] # все которым он поставил дизлайк

# тут фильтруем топ-10 по ключевым словам, которые не понравились в прошлых анекдотах
tokenizer = NLTKWordTokenizer()
unliked_tok = [tokenizer.tokenize(x.lower()) for x in unliked]
top_jokes_tok = [tokenizer.tokenize(x.lower()) for x in top_jokes]

lemmatizer = WordNetLemmatizer()

russian_stopwords = stopwords.words("russian")
unliked_tokens = []
for row in unliked_tok:
    row_tokens = []
    for token in row:
        if token not in russian_stopwords and token not in punctuation:
            row_tokens.append(lemmatizer.lemmatize(token))
    unliked_tokens.append(' '.join(row_tokens))

top_tokens = []
for row in top_jokes_tok:
    row_tokens = []
    for token in row:
        if token not in russian_stopwords and token not in punctuation:
            row_tokens.append(lemmatizer.lemmatize(token))
    top_tokens.append(row_tokens)


tf_idf = TfidfVectorizer()
unliked_array = tf_idf.fit_transform(unliked_tokens)
threshold = 0.2 # тут его возможно надо будет поднять
important_words = [word for word, score in zip(tf_idf.get_feature_names_out(), unliked_array.toarray()[0]) if score >= threshold]
important_set = set(important_words)
counts = []
for joke in top_tokens:
    local_count = 0
    for token in joke:
        if token in important_words:
            local_count += 1
    counts.append(local_count)

print(top_jokes[np.argmin(counts)]) # это шутка, которую выводим